# Rock concert planning

The promoters of a rock concert in Madison must perform the tasks
shown in the GAMS code below before the concert can be held:

In [1]:
# Load the gams extension
%load_ext gams_magic

In [2]:
%%gams
set activity /
        A "Find Site",
        B "Find Engineers",
        C "Hire Opening Act",
        D "Set Radio and TV Ads",
        E "Set Up Ticket Agents",
        F "Prepare Electronics",
        G "Print Advertising",
        H "Set up Transportation",
        I "Rehearsals",
        J "Last-Minute Details"
/;

alias (activity,ai,aj);

set pred(ai,aj) "ai preceeds aj" /
        A. (B,C,E) 
        B . F
        C . (D,G,H)
        (F,H) . I
        I . J
/;

parameter duration(activity) "in days" /
        A       3,      B       2
        C       6,      D       2
        E       3,      F       3
        G       5,      H       1
        I       1.5,    J       2
/;

If any increase in the duration of an activity increases the time
needed to complete the project, then the activity is called a critical
activity.  We will find the critical activities in this project in two
different ways.

### Method 1
Set up a linear program to find the project duration (that is the
minimum number of days needed to prepare for the concert).  Determine the critical tasks by inspecting the multipliers of this problem!   Make sure you store and print out the critical tasks.

In [3]:
%%gams
free variable T 'time to completion';
nonnegative variable S(activity) 'starting time for activity';

equations ctime(activity), ptime(ai,aj);
ctime(activity).. T =G= S(activity) + duration(activity);
ptime(pred(ai,aj)).. S(ai) + duration(ai) =L= S(aj);

model cpm /ctime,ptime/;
solve cpm using LP minimizing T;

parameters critical(*), T_star;
critical(activity) = S.m(activity);
T_star = T.l;
display S.l, T_star, critical;


,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),14.0,20,11,LP,CPLEX,0.016


In [4]:
%gams_pull critical
print("critical arcs:", *critical)

critical arcs: ('A', 1.0)


### Method 2
Now use two
other linear programs to find the early event times (the earliest
possible starting time for an activity) and the late event
times (the latest possible time to start the activity without delaying
project completion time).  Then construct the critical activities by seeing which early
event times are the same as the corresponding late event times.
Make sure your print out includes a listing of the
early and late event times, and the critical activities.

In [5]:
%%gams
free variable TS 'sum of starting times';

equations Tmax, Tmin, TSdef;
Tmax.. T_star =G= T;
Tmin.. T_star =L= T;
TSdef.. TS =E= sum(ai, S(ai))

model mTmax /ctime,ptime,Tmax,TSdef/;
model mTmin /ctime,ptime,Tmin,TSdef/;

parameters
    leTime(activity) 'latest   possible starting event time without delaying project completion time',
    eeTime(activity) 'earliest possible starting event time without delaying project completion time';

solve mTmax using LP maximizing TS;
leTime(activity) = S.l(activity);

solve mTmax using LP minimizing TS;
eeTime(activity) = S.l(activity);

loop(activity$(ord(activity)>1 ),
if (eeTime(activity) = leTime(activity),
critical(activity) = eeTime(activity);
));

display eeTime, leTime, critical;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),80.0,22,12,LP,CPLEX,0.11
1,Normal (1),Optimal Global (1),62.5,22,12,LP,CPLEX,0


In [6]:
%gams_pull eeTime leTime critical
#display(eeTime,leTime,critical)
print("critical arcs:", *critical)
print("early event times:", *eeTime, sep='\n')
print("late event times:", *leTime, sep='\n')

critical arcs: ('A', 1.0) ('C', 3.0) ('G', 9.0)
early event times:
('B', 3.0)
('C', 3.0)
('D', 9.0)
('E', 3.0)
('F', 5.0)
('G', 9.0)
('H', 9.0)
('I', 10.0)
('J', 11.5)
late event times:
('B', 5.5)
('C', 3.0)
('D', 12.0)
('E', 11.0)
('F', 7.5)
('G', 9.0)
('H', 9.5)
('I', 10.5)
('J', 12.0)


### Which of these two methods is more reliable?

The second method seems more reliable. The first method did not identify the critical events C and G. 